In [2]:
!pip install sahi
!pip install ultralytics

import torch
import cv2
import numpy as np
from ultralytics import YOLO

# -----------------------------
# 1️⃣ Model Paths
# -----------------------------
model_paths = {
    "yolov10n": "https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov10n.pt",
    "yolov9t": "https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov9t.pt",
    "yolov8n": "https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt"
}

# Load models
models = {name: YOLO(path) for name, path in model_paths.items()}

# -----------------------------
# 2️⃣ Input & Output Path
# -----------------------------
image_path = "/content/drive/MyDrive/Human_Detection_Test.jpg"   # 👈 change to your image
output_path = "/content/drive/MyDrive/fused_output.jpg"

# -----------------------------
# 3️⃣ IOU Function
# -----------------------------
def iou(box1, box2):
    x1, y1, x2, y2 = box1[:4]
    x1g, y1g, x2g, y2g = box2[:4]

    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2, y2g)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    return inter_area / float(box1_area + box2_area - inter_area + 1e-6)

# -----------------------------
# 4️⃣ Fusion Function
# -----------------------------
def fuse_detections(detections_list, iou_thresh=0.5):
    fused = []

    for detections in detections_list:
        for det in detections:
            if all(iou(det, existing) < iou_thresh for existing in fused):
                fused.append(det)

    return fused

# -----------------------------
# 5️⃣ Run Detection
# -----------------------------
img = cv2.imread(image_path)

all_detections = []

for name, model in models.items():
    results = model.predict(img, conf=0.25, iou=0.45, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()
    all_detections.append(detections)

# Fuse predictions
fused_detections = fuse_detections(all_detections)

# -----------------------------
# 6️⃣ Draw Final Boxes
# -----------------------------
for det in fused_detections:
    x1, y1, x2, y2, conf, cls = det

    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(
        img,
        f"Class {int(cls)}: {conf:.2f}",
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        2
    )

# Save result
cv2.imwrite(output_path, img)

print(f"Fused detection saved at: {output_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov9t.pt... Remote end closed connection without response
Fused detection saved at: /content/drive/MyDrive/fused_output.jpg


In [3]:
!pip install sahi ultralytics

import cv2
import numpy as np
from sahi.predict import get_sliced_prediction
from sahi.models.ultralytics import UltralyticsDetectionModel

# -----------------------------
# 1️⃣ Model Paths
# -----------------------------
model_paths = {
    "yolov10n": "yolov10n.pt",
    "yolov9t": "yolov9t.pt",
    "yolov8n": "yolov8n.pt"
}

# Load SAHI detection models
models = {
    name: UltralyticsDetectionModel(
        model_path=path,
        confidence_threshold=0.25,
        device="cuda"  # change to "cpu" if needed
    )
    for name, path in model_paths.items()
}

# -----------------------------
# 2️⃣ Input / Output
# -----------------------------
image_path = "/content/drive/MyDrive/Human_Detection_Test.jpg"
output_path = "/content/drive/MyDrive/multimodel_sahi_human.jpg"

# -----------------------------
# 3️⃣ IoU Function
# -----------------------------
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2, y2g)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    return inter_area / float(box1_area + box2_area - inter_area + 1e-6)

# -----------------------------
# 4️⃣ Fusion Function
# -----------------------------
def fuse_boxes(detections, iou_thresh=0.5):
    fused = []
    for det in detections:
        box = det[:4]
        score = det[4]

        if all(iou(box, existing[:4]) < iou_thresh for existing in fused):
            fused.append(det)

    return fused

# -----------------------------
# 5️⃣ Run SAHI on All Models
# -----------------------------
all_detections = []

for name, model in models.items():
    result = get_sliced_prediction(
        image=image_path,
        detection_model=model,
        slice_height=512,
        slice_width=512,
        overlap_height_ratio=0.2,
        overlap_width_ratio=0.2,
    )

    for obj in result.object_prediction_list:
        if obj.category.id == 0:  # Human class only
            bbox = obj.bbox
            score = obj.score.value

            all_detections.append([
                bbox.minx,
                bbox.miny,
                bbox.maxx,
                bbox.maxy,
                score
            ])

# -----------------------------
# 6️⃣ Apply Fusion
# -----------------------------
fused_detections = fuse_boxes(all_detections)

# -----------------------------
# 7️⃣ Draw Final Results
# -----------------------------
img = cv2.imread(image_path)

for det in fused_detections:
    x1, y1, x2, y2, score = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(
        img,
        f"Human: {score:.2f}",
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 255, 0),
        2
    )

cv2.imwrite(output_path, img)

print("Multi-Model + SAHI human detection saved at:", output_path)

Performing prediction on 8 slices.
Performing prediction on 8 slices.
Performing prediction on 8 slices.
Multi-Model + SAHI human detection saved at: /content/drive/MyDrive/multimodel_sahi_human.jpg
